In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
from datetime import datetime
import json

# Preprocess COF Data

In [2]:
COF_data_path = '/Users/kieran/iMoDELS-supplements/data/raw-data/everything.csv'
COF_data = pd.read_csv(COF_data_path, index_col=0)
COF_data = COF_data.drop(['backbone'],axis=1)

molecules = glob.glob('../terminal_group_screening/src/util/molecules/*.xyz')
molecules = list(set(molecules))
im_mols = {}
for m in molecules:
    mol_name = m.split('/')[-1].split('.')[0]
    if 'ch3' in mol_name:
        mol_name = mol_name.split('-')[0]
        
COD_data = COF_data.dropna()
msk = np.random.rand(len(COF_data)) < 0.8

train, test = COF_data[msk] , COF_data[~msk]
train_X, train_y = train, train['COF']
train_X = train_X.drop(['COF'],axis=1)
test_X, test_y = test, test['COF']

# Get QM9 property prediction working

In [12]:
%cd egnn

import sys
sys.path.insert(1, 'egnn/models/egnn_clean')
sys.path.insert(1, 'egnn/qm9')
import qm9
from qm9 import dataset
from easydict import EasyDict as edict
from qm9 import utils as qm9_utils
from qm9.models import EGNN
import torch
from torch import nn, optim

args = edict({'batch_size':8, 'num_workers':2, 'dataset':'qm9', 'datadir':'/Users/kieran/forward-pred/qm9/data/qm9/qm9',
            'filter_n_atoms':None, 'remove_h':True, 'include_charges':True, 'shuffle': True, 'property': 'alpha', 'nf':128,
             'n_layers':7, 'attention':True, 'node_attr':0, 'lr':1e-3, 'weight_decay':1e-16, 'epochs':10, 'charge_power':2,
             'test_interval':1, 'log_interval':20})
args.cuda = False
device = torch.device("cuda" if args.cuda else "cpu")
dtype = torch.float32
dataloaders, charge_scale = dataset.retrieve_dataloaders(args.batch_size)

# compute mean and mean absolute deviation
meann, mad = qm9_utils.compute_mean_mad(dataloaders, args.property)

model = EGNN(in_node_nf=15, in_edge_nf=0, hidden_nf=args.nf, device=device, n_layers=args.n_layers, coords_weight=1.0,
             attention=args.attention, node_attr=args.node_attr)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)
loss_l1 = nn.L1Loss()

[Errno 2] No such file or directory: 'egnn'
/Users/kieran/forward-pred/egnn


In [13]:
def train(epoch, loader, partition='train'):
    lr_scheduler.step()
    res = {'loss': 0, 'counter': 0, 'loss_arr':[]}
    for i, data in enumerate(loader):
        if partition == 'train':
            model.train()
            optimizer.zero_grad()

        else:
            model.eval()

        batch_size, n_nodes, _ = data['positions'].size()
        atom_positions = data['positions'].view(batch_size * n_nodes, -1).to(device, dtype)
        atom_mask = data['atom_mask'].view(batch_size * n_nodes, -1).to(device, dtype)
        edge_mask = data['edge_mask'].to(device, dtype)
        one_hot = data['one_hot'].to(device, dtype)
        charges = data['charges'].to(device, dtype)
        nodes = qm9_utils.preprocess_input(one_hot, charges, args.charge_power, charge_scale, device)

        nodes = nodes.view(batch_size * n_nodes, -1)
        # nodes = torch.cat([one_hot, charges], dim=1)
        edges = qm9_utils.get_adj_matrix(n_nodes, batch_size, device)
        label = data[args.property].to(device, dtype)

        pred = model(h0=nodes, x=atom_positions, edges=edges, edge_attr=None, node_mask=atom_mask, edge_mask=edge_mask,
                     n_nodes=n_nodes)

        if partition == 'train':
            loss = loss_l1(pred, (label - meann) / mad)
            loss.backward()
            optimizer.step()
        else:
            loss = loss_l1(mad * pred + meann, label)

        res['loss'] += loss.item() * batch_size
        res['counter'] += batch_size
        res['loss_arr'].append(loss.item())

        prefix = ""
        if partition != 'train':
            prefix = ">> %s \t" % partition

        if i % args.log_interval == 0:
            print(prefix + "Epoch %d \t Iteration %d \t loss %.4f" % (epoch, i, sum(res['loss_arr'][-10:])/len(res['loss_arr'][-10:])))
    return res['loss'] / res['counter']

In [14]:
res = {'epochs': [], 'losess': [], 'best_val': 1e10, 'best_test': 1e10, 'best_epoch': 0}

for epoch in range(0, args.epochs):
    train(epoch, dataloaders['train'], partition='train')
    if epoch % args.test_interval == 0:
        val_loss = train(epoch, dataloaders['valid'], partition='valid')
        test_loss = train(epoch, dataloaders['test'], partition='test')
        res['epochs'].append(epoch)
        res['losess'].append(test_loss)

        if val_loss < res['best_val']:
            res['best_val'] = val_loss
            res['best_test'] = test_loss
            res['best_epoch'] = epoch
        print("Val loss: %.4f \t test loss: %.4f \t epoch %d" % (val_loss, test_loss, epoch))
        print("Best: val loss: %.4f \t test loss: %.4f \t epoch %d" % (res['best_val'], res['best_test'], res['best_epoch']))


    json_object = json.dumps(res, indent=4)
    with open(args.outf + "/" + args.exp_name + "/losess.json", "w") as outfile:
        outfile.write(json_object)

/Users/kieran/opt/miniconda3/envs/forward/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 0 	 Iteration 0 	 loss 0.3911
Epoch 0 	 Iteration 20 	 loss 0.7245
Epoch 0 	 Iteration 40 	 loss 0.6656
Epoch 0 	 Iteration 60 	 loss 0.4734
Epoch 0 	 Iteration 80 	 loss 0.5782
Epoch 0 	 Iteration 100 	 loss 0.5039
Epoch 0 	 Iteration 120 	 loss 0.4655
Epoch 0 	 Iteration 140 	 loss 0.5755
Epoch 0 	 Iteration 160 	 loss 0.6894
Epoch 0 	 Iteration 180 	 loss 0.4543
Epoch 0 	 Iteration 200 	 loss 0.5118
Epoch 0 	 Iteration 220 	 loss 0.5031
Epoch 0 	 Iteration 240 	 loss 0.3978
Epoch 0 	 Iteration 260 	 loss 0.4294
Epoch 0 	 Iteration 280 	 loss 0.5825
Epoch 0 	 Iteration 300 	 loss 0.5387
Epoch 0 	 Iteration 320 	 loss 0.4653
Epoch 0 	 Iteration 340 	 loss 0.4446
Epoch 0 	 Iteration 360 	 loss 0.3655
Epoch 0 	 Iteration 380 	 loss 0.3785
Epoch 0 	 Iteration 400 	 loss 0.4293
Epoch 0 	 Iteration 420 	 loss 0.4055
Epoch 0 	 Iteration 440 	 loss 0.2688
Epoch 0 	 Iteration 460 	 loss 0.4606
Epoch 0 	 Iteration 480 	 loss 0.3924
Epoch 0 	 Iteration 500 	 loss 0.4249
Epoch 0 	 Iteratio

NameError: name 'json' is not defined

## Basic Example

In [2]:
import egnn_clean as eg
import torch

# Dummy parameters
batch_size = 8
n_nodes = 4
n_feat = 1
x_dim = 3

# Dummy variables h, x and fully connected edges
h = torch.ones(batch_size * n_nodes, n_feat)
x = torch.ones(batch_size * n_nodes, x_dim)
edges, edge_attr = eg.get_edges_batch(n_nodes, batch_size)

# Initialize EGNN
egnn = eg.EGNN(in_node_nf=n_feat, hidden_nf=32, out_node_nf=1, in_edge_nf=1)

# Run EGNN
h, x = egnn(h, x, edges, edge_attr)

In [ ]:
%run -u main_qm9.py --num_workers 2 --lr 5e-4 --property alpha --exp_name exp_1_alpha